In [1]:
# -- coding: utf-8 --
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import math
import json
import tqdm
import os
import random
import shutil
import time
import torch
import torchvision
from torch import nn, optim
from PIL import Image
from torch.utils.data import Dataset

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
# data path
photo_path = '/Users/imchengliang/Downloads/DS-Project/photo/'
train_path = '/Users/imchengliang/Downloads/DS-Project/b_w_image/'
json_path = '/Users/imchengliang/Downloads/DS-Project/json-data/'
train_path = '/Users/imchengliang/Downloads/DS-Project/b_w_image/'
data_path = '/Users/imchengliang/Downloads/DS-Project/data/'
data_path_1 = '/Users/imchengliang/Downloads/DS-Project/train_0/'

## Save coordinate data into csv

In [6]:
def generate_train_df():
    file_list = sorted(os.listdir(json_path))
    info_list = []
    for f in file_list:
        path = json_path + '/' + f
        p = f.split(".")
        p_path = photo_path + p[0] + '.png'
        with open(path) as data:
            dict1 =  json.load(data)
            for i in range(len(dict1['shapes'])):
                info = {}
                info['number'] = p[0]
                info['jsonname'] = path
                info['photoname'] = p_path
                info['width'] = dict1['imageWidth']
                info['height'] = dict1['imageHeight']
                info['class'] = dict1['shapes'][i]['label']
                info['xmin'] = dict1['shapes'][i]['points'][0][0]
                info['ymin'] = dict1['shapes'][i]['points'][0][1]
                info['xmax'] = dict1['shapes'][i]['points'][1][0]
                info['ymax'] = dict1['shapes'][i]['points'][1][1]
                info_list.append(info)
    return pd.DataFrame(info_list)


In [7]:
df_train = generate_train_df()
df_train.to_csv('data.csv', index=False)

(1708, 10)


## Switch the image into black and white

In [3]:
def preprocess(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # binary conversions
    ret, threshold = cv2.threshold(gray, 160, 255, cv2.THRESH_BINARY)
    #kernel = np.ones((2, 2), np.uint8)
    # morphological operations:
    #erosion = cv2.erode(threshold, kernel, iterations = 1)
    #dilation = cv2.dilate(threshold, kernel,iterations = 1)
    #opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel)
    #closing = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, kernel)
    # rescale the size
    #new_width  = 700
    #new_height = 1000
    #resized_image = cv2.resize(threshold, (new_width, new_height))
    #cv2.imshow("cvtColor2gray", closing)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #plt.imshow(closing, 'gray')
    #plt.show()
    a = path.split('/')[-1]
    cv2.imwrite('/Users/imchengliang/Downloads/DS-Project/b_w_image/'+a, threshold, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [8]:
file_list = sorted(os.listdir(photo_path))
for i in file_list:
    preprocess('/Users/imchengliang/Downloads/DS-Project/photo/'+i)

## Save the coordination data into a dictionary

In [ ]:
file = pd.read_csv('data.csv')
id = file['number'].unique()
coordinates = {}
for i in id:
    a = file.loc[file['number']==i]
    coordinate = []
    #ratio_w = new_width / a.iloc[0]['width']
    #ratio_h = new_height / a.iloc[0]['height'] 
    for index, row in a.iterrows():
        b = []
        b.append(row['xmin'])
        b.append(row['ymin'])
        b.append(row['xmax'])
        b.append(row['ymax'])
        b.append(row['class'])
        coordinate.append(b)
    coordinates[int(i)] = coordinate

In [24]:
with open('coordinates.json', 'w') as f:
    json.dump(coordinates, f)

In [16]:
with open('coordinates.json') as f:
    coordinates_l = json.load(f)

## Cut the handwritten part form the image

In [ ]:
cut_shape = {}
for i in coordinates_l.keys():
    print(i)
    img_path = photo_path+i+'.png'
    im = cv2.imread(img_path)
    a = 1
    for j in coordinates_l[i]:
        name = i+'_'+str(a)
        # y first, x second
        ims = im[math.ceil(j[1]):math.ceil(j[3]), math.ceil(j[0]):math.ceil(j[2])]
        cut_shape[name] = [ims.shape[0], ims.shape[1]]
        cv2.imwrite('/Users/imchengliang/Downloads/DS-Project/cut_image/'+i+'_'+str(a)+'.png', ims)
        a += 1

## Save the cut-handwritten part shape information into a dictionary

In [36]:
with open('cut_shape.json', 'w') as f:
    json.dump(cut_shape, f)

In [101]:
with open('cut_shape.json') as f:
    cut_shape = json.load(f)

## Cut and reshape the training data

In [21]:
def pre_data(path1, path2, j):
    img = cv2.imread(path1+j+'.png')
    kernel = np.ones((2, 2), np.uint8)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    if (opening.shape[0]>=opening.shape[1] and (opening.shape[0]/opening.shape[1])<=2.07) or (opening.shape[0]<=opening.shape[1] and (opening.shape[1]/opening.shape[0])<=2.07):
        resized_image = cv2.resize(opening, (227, 227))
        cv2.imwrite(path2+j+'_'+'.png', resized_image)
    elif opening.shape[0]>opening.shape[1] and (opening.shape[0]/opening.shape[1])>2.07:
        n = int(opening.shape[0] / opening.shape[1])
        dis_h = int(np.floor(opening.shape[0] / n))
        for i in range(n):
            a = i+1
            sub = opening[i*dis_h:(i+1)*dis_h, 0:opening.shape[1]]
            resized_image = cv2.resize(sub, (227, 227))
            cv2.imwrite(path2+j+'_'+str(a)+'.png', resized_image)
    elif opening.shape[0]<opening.shape[1] and (opening.shape[1]/opening.shape[0])>2.07:
        n = int(opening.shape[1] / opening.shape[0])
        dis_w = int(np.floor(opening.shape[1] / n))
        for i in range(n):
            a = i+1
            sub = opening[0:opening.shape[0], i*dis_w:(i+1)*dis_w]
            resized_image = cv2.resize(sub, (227, 227))
            cv2.imwrite(path2+j+'_'+str(a)+'.png', resized_image)
    print(j)


In [105]:
for j in cut_shape.keys():
    pre_data(j)

183786_1
183786_2
183786_3
183844_1
183845_1
183846_1
183846_2
183846_3
183846_4
183846_5
183846_6
183846_7
183847_1
183855_1
183855_2
183855_3
183862_1
183862_2
183862_3
183867_1
183867_2
183868_1
183868_2
183869_1
183869_2
183874_1
183874_2
183876_1
183876_2
183876_3
183876_4
183876_5
183876_6
183876_7
183878_1
183878_2
183878_3
183878_4
183878_5
183881_1
183919_1
183942_1
183942_2
183942_3
183943_1
183943_2
183971_1
183971_2
183971_3
184010_1
184011_1
184012_1
184015_1
184015_2
184018_1
184064_1
184064_2
184064_3
184065_1
184065_2
184067_1
184068_1
184068_2
184076_1
184076_2
184078_1
184078_2
184078_3
184078_4
184078_5
184078_6
184078_7
184078_8
184078_9
184101_1
184101_2
184102_1
184102_2
184104_1
184105_1
184105_2
184105_3
184127_1
184127_2
184127_3
184127_4
184127_5
184130_1
184130_2
184130_3
184130_4
184130_5
184145_1
184145_2
184145_3
184145_4
184146_1
184148_1
184148_2
184156_1
184156_2
184156_3
184157_1
184157_2
184157_3
184159_1
184159_2
184159_3
184251_1
184251_2
184251_3
1

## Divide data into training and validation

In [37]:
train_dir = './train/'
valid_dir = './valid/'
test_dir = './test_1/'
train_pct = 0.8
valid_pct = 0.1
test_pct = 0.1

imgs = os.listdir(data_path_1) 
imgs = list(filter(lambda x: x.endswith('.png'), imgs)) 
random.shuffle(imgs)  
img_count = len(imgs)  
train_point = int(img_count * train_pct)  
valid_point = int(img_count * (train_pct + valid_pct))  

for i in range(img_count):
    if i < train_point:  
        out_dir = train_dir
    elif i < valid_point:  
        out_dir = valid_dir
    
    target_path = out_dir + str(imgs[i]) 
    src_path = data_path_1 + str(imgs[i])  
    shutil.copy(src_path, target_path)  

print('train:{}, valid:{}'.format(train_point, valid_point-train_point))


train:2296, valid:574


## Save the train, valid, test data dir in a txt for model-training

In [47]:
train_txt_path = './train.txt'
valid_txt_path = './valid.txt'
test_txt_path = './test.txt'
path = './'

def gen_txt(txt_path, img_dir):
    f = open(txt_path, 'w')
    for s_dir in sorted(os.listdir(img_dir)):
        i_dir = os.path.join(img_dir, s_dir)
        #print(i_dir.split('_'))
        if i_dir.split('_')[2] == '0':
            line = i_dir + ' ' + '0' + '\n'
        else:
            line = i_dir + ' ' + '1' + '\n'

        f.write(line)
        
    f.close()

In [48]:
gen_txt(valid_txt_path, valid_dir)
gen_txt(train_txt_path, train_dir)
gen_txt(test_txt_path, test_dir)

## Check the maximum and minium of box size

In [3]:
file = pd.read_csv('data.csv')
x, y, xy = [], [], []
for i in range(len(file['number'])):
    x.append(file['xmax'][i]-file['xmin'][i])
    y.append(file['ymax'][i]-file['ymin'][i])
    xy.append((file['xmax'][i]-file['xmin'][i])*(file['ymax'][i]-file['ymin'][i]))
print(max(x), min(x), max(y), min(y), max(xy), min(xy))

3528.0 41.176470588235304 3703.703703703703 33.33333333333337 7492408.329560887 1614.7635524798125


## Create training samples of non-handwritten part

In [30]:
with open('coordinates.json') as f:
    coordinates = json.load(f)

with open ('new_test_coordinates.json') as f:
    new_test_coordinates = json.load(f)

train_0 = {}
for i in new_test_coordinates.keys():
    train_0[i] = []

In [29]:
def get_IOU(bb1, bb2):
    # Finding intersection
    a, b = bb2[0]+bb2[2], bb2[1]+bb2[3]
    x_left = max(bb1[0], bb2[0])
    y_bottom = max(bb1[1], bb2[1])
    x_right = min(bb1[2], a)
    y_top = min(bb1[3], b)
    
    # Checking whether there is any intersection avaiable
    # If not, return area of intersection as 0.
    if (x_left > x_right) or (y_bottom > y_top):
        return 0.0

    area_intersect = (x_right - x_left) * (y_top - y_bottom)
    
    # To bring into percentage, calculate area of each bounding box
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = bb2[2] * bb2[3]
    total_area = bb1_area + bb2_area - area_intersect
    iou = area_intersect / total_area
    
    return iou

In [31]:
for i in coordinates.keys():
    for k in new_test_coordinates[i]:
        if len(train_0[i]) <= 3:
            iou = 0
            for j in coordinates[i]:
                iou += get_IOU(j, k)
            if iou/len(j) == 0:
                train_0[i].append(k)

In [32]:
with open('0.json', 'w') as f:
    json.dump(train_0, f)

In [ ]:
cut_shape_0 = {}
for i in train_0.keys():
    print(i)
    img_path = train_path+i+'.png'
    im = cv2.imread(img_path)
    a = 0
    for j in train_0[i]:
        b, c = j[1]+j[3], j[0]+j[2]
        if j[1] < 0:
            j[1] = 0
        if j[0] < 0:
            j[0] = 0
        name = i+'_'+str(a)
        b, c = j[1]+j[3], j[0]+j[2]
        ims = im[j[1]:b, j[0]:c]
        cut_shape_0[name] = [int(j[2]), int(j[3])]
        cv2.imwrite('./train_1/'+i+'_0_'+str(a)+'.png', ims)
        a += 1

In [34]:
with open('cut_shape_0.json', 'w') as f:
    json.dump(cut_shape_0, f)

In [ ]:
path1 = './train_1/'
path2 = './train_0/'
for j in cut_shape_0.keys():
    name = j.split('_')[0]+'_0_'+j.split('_')[-1]
    pre_data(path1, path2, name)